# Function calling

https://platform.openai.com/docs/guides/function-calling

Function Calling은 OpenAI의 GPT 모델이 특정 작업을 수행할 수 있도록 함수 호출을 지원하는 기능이다.

이 기능을 활용하면 GPT 모델이 단순한 텍스트 생성뿐만 아니라 더 복잡한 작업도 자동으로 처리할 수 있다.

Function Calling을 통해 다음과 같은 작업을 수행할 수 있다:

1. **API 호출**: 외부 API를 호출하여 실시간 데이터를 가져오거나 특정 작업을 수행할 수 있다.  
   예) 날씨 정보 조회, 금융 데이터 조회

2. **데이터 처리**: 특정 함수를 호출하여 데이터 처리 작업을 수행할 수 있다.  
   예) 텍스트 데이터 분석, 통계 계산

3. **자동화된 작업**: 여러 단계의 작업을 자동화하여 수행할 수 있다.  
   예) 사용자가 요청한 내용을 기반으로 보고서 생성, 이메일 작성

**주요 특징**

- **함수 정의**: 사전에 정의된 함수나 API 엔드포인트를 GPT 모델에 통합할 수 있다.
- **자동 호출**: 사용자가 명시적으로 요청하지 않아도, GPT 모델이 컨텍스트를 이해하고 자동으로 적절한 함수를 호출할 수 있다.
- **입출력 처리**: 함수의 입력값을 자동으로 생성하고, 함수 호출 후 반환된 결과를 적절히 처리하여 사용자에게 전달할 수 있다.

OPENWEATHER API
- https://openweathermap.org/

## Function 정의

In [ ]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
OPENWEATHER_API_KEY = userdata.get('OPENWEATHER_API_KEY')

In [ ]:
import requests
import json

def get_current_weather(city, units):
  url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units={units}'
  # print(url)

  response = requests.get(url) # 웹요청 requests
  data = response.json() # response content(json) 반환
  # print(data)

  # llm에 전달할 날씨 정보
  if response.status_code == 200:
    weather_description = data['weather'][0]['description']
    temperature = data['main']['temp']
    humidity = data['main']['humidity']
    weather_info = {
        'location': city,
        'weather': weather_description,
        'temperature': temperature,
        'humidity': humidity,
        'unit': 'celsius' if units == 'metric' else 'fahrenheit'
    }
  else:
    # api 요청 실패한 경우
    weather_info = {
        'location': city,
        'weather': 'Not Found',
        'temperature': 'Not Found',
        'humidity': 'Not Found',
        'unit': 'Not Found'
    }

  return json.dumps(weather_info)


get_current_weather('서초동', 'metric')

'{"location": "\\uc11c\\ucd08\\ub3d9", "weather": "mist", "temperature": 21.89, "humidity": 94, "unit": "celsius"}'

## LLM 요청 흐름

In [ ]:
from openai import OpenAI

# LLM에게 각 function을 설명하는 메타정보
tools = [{
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "주어진 도시에 대해 현재 날씨 정보를 가져옵니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "날씨 정보를 얻고자하는 도시 이름(영문)"
                },
                "units": {
                    "type": "string",
                    "enum": ['metric', 'imperial'],
                    "description": "날씨 정보에 대한 단위설정. 섭씨를 사용하는 경우 metric을, 화씨를 사용하는 경우 imperial을 선택해주세요."
                }
            },
            "required": [
                "city"
            ],
            "additionalProperties": False
        }
    }
}]

messages = [
    {'role': 'system', 'content': '당신은 친절한 챗봇으로, 사용자의 요청에 답변을 제공하세요.'},
    {'role': 'user', 'content': '오늘 서울 날씨는 어떤가요?'}
]

client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
    tools=tools
)
response

ChatCompletion(id='chatcmpl-BmXULd3OGGJ00r8i5kzX33pmEGyyZ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ZNzbEWaApV1aSACQdONum2YR', function=Function(arguments='{"city":"Seoul","units":"metric"}', name='get_current_weather'), type='function')]))], created=1750908173, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_62a23a81ef', usage=CompletionUsage(completion_tokens=20, prompt_tokens=135, total_tokens=155, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [ ]:
# llm으로부터 응답받은 function_call 정보
response_message = response.choices[0].message
print(response_message)
tool_calls = response_message.tool_calls
tool_calls

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ZNzbEWaApV1aSACQdONum2YR', function=Function(arguments='{"city":"Seoul","units":"metric"}', name='get_current_weather'), type='function')])


[ChatCompletionMessageToolCall(id='call_ZNzbEWaApV1aSACQdONum2YR', function=Function(arguments='{"city":"Seoul","units":"metric"}', name='get_current_weather'), type='function')]

In [ ]:
# 함수목록에서 해당함수를 찾고 실행 - 실행결과를 다시 llm 질의

# 실행가능한 함수목록
available_functions = {
    'get_current_weather': get_current_weather
}

# llm(assistant): function_call 응답
messages.append(response_message) # 'role': 'assistant', 'tool_calls': ...

for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments) # json str -> dict
    # print(function_to_call)
    # print(function_args)
    function_response = function_to_call(**function_args)
    # print(function_response)

    messages.append({
        'role': 'tool',
        'tool_call_id': tool_call.id,
        'name': function_name,
        'content': function_response
    })

messages


[{'role': 'system', 'content': '당신은 친절한 챗봇으로, 사용자의 요청에 답변을 제공하세요.'},
 {'role': 'user', 'content': '오늘 서울 날씨는 어떤가요?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ZNzbEWaApV1aSACQdONum2YR', function=Function(arguments='{"city":"Seoul","units":"metric"}', name='get_current_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_ZNzbEWaApV1aSACQdONum2YR',
  'name': 'get_current_weather',
  'content': '{"location": "Seoul", "weather": "broken clouds", "temperature": 23.76, "humidity": 94, "unit": "celsius"}'}]

In [ ]:
# 최종 llm 응답
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
)
response.choices[0].message.content

'오늘 서울의 날씨는 구름이 조금 끼어 있는 상태이며, 현재 기온은 약 23.8도입니다. 습도는 94%로 다소 높은 편입니다. 외출 시 가벼운 겉옷을 챙기시는 것이 좋을 것 같습니다!'

In [ ]:
# toll_call을 포함한 함수
def run_conversation(prompt):
  messages = [
      {'role': 'system', 'content': '당신은 친절한 챗봇으로, 사용자의 요청에 답변을 제공하세요.'},
      {'role': 'user', 'content': prompt}
  ]

  client = OpenAI(api_key=OPENAI_API_KEY)

  # 첫번째 LLM 요청
  response = client.chat.completions.create(
      model='gpt-4o-mini',
      messages=messages,
      tools=tools
  )

  # 첫번째 LLM 응답 확인
  response_message = response.choices[0].message
  tool_calls = response_message.tool_calls

  # tool_call이 필요한 경우와 아닌 경우 분기
  if tool_calls:
    # n건의 함수 호출
    messages.append(response_message) # 'role': 'assistant', 'tool_calls': ...

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments) # json str -> dict
        function_response = function_to_call(**function_args)

        messages.append({
            'role': 'tool',
            'tool_call_id': tool_call.id,
            'name': function_name,
            'content': function_response
        })

    # 두번째 LLM 요청
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )

  return response.choices[0].message.content

print(run_conversation('밥 먹었니?'))
print(run_conversation('오늘 날씨 어때?'))

저는 음식이 필요하지 않지만, 당신은 어떤 식사를 하셨나요? 궁금하네요!
어떤 도시에 대한 날씨 정보를 원하시나요? 도시 이름을 알려주시면 해당 날씨를 확인해드리겠습니다.
